In [46]:
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship
from sqlalchemy import Integer, String, DateTime, ForeignKey, ARRAY
import datetime
from typing import Annotated
from enum import StrEnum

str_256 = Annotated[str, mapped_column(String(256))]

class UserRole(StrEnum):
    """Enumeration for user roles."""
    ADMIN = "admin"
    USER = "user"

class Base(DeclarativeBase):
    pass


class User(Base):
    """Represents a user in the system."""
    __tablename__ = "users"

    id: Mapped[int] = mapped_column(primary_key=True, doc="Unique user identifier")
    username: Mapped[str] = mapped_column(String(50), unique=True, nullable=False, doc="Username of the user")
    email: Mapped[str] = mapped_column(String(100), unique=True, nullable=False, doc="User's email address")
    created_at: Mapped[datetime.datetime] = mapped_column(default=datetime.datetime.utcnow, doc="Datetime when the user was created")
    user_role: Mapped[UserRole] = mapped_column(server_default=UserRole.USER, doc="Role of the user in the system")
    posts: Mapped[list["Post"]] = relationship(back_populates="author", doc="List of posts created by the user")


class Post(Base):
    __tablename__ = "posts"

    id: Mapped[int] = mapped_column(primary_key=True, doc="Unique post identifier")
    title: Mapped[str] = mapped_column(String(100), nullable=False, doc="Title of the post")
    content: Mapped[str_256] = mapped_column(doc="Main content of the post")
    user_id: Mapped[int] = mapped_column(ForeignKey("users.id"), doc="ID of the user who created the post")
    author: Mapped["User"] = relationship(back_populates="posts", doc="User who authored the post")
    test: Mapped[list[str]] = mapped_column(ARRAY(String), doc="Test field for demonstration purposes")


In [47]:
User.__mapper__.c.get("h", None)

In [48]:
User.__name__
User.__doc__
User.__tablename__

'users'

In [49]:
for con in User.__table__.constraints:
    print(len(con.columns))

1
1
1


In [50]:
from collections import defaultdict
constraints = defaultdict(set)
for constraint in User.__table__.constraints:
    for col in constraint.columns:
        constraints[constraint.__class__.__name__].add(col.name)
constraints

defaultdict(set,
            {'PrimaryKeyConstraint': {'id'},
             'UniqueConstraint': {'email', 'username'}})

In [68]:
columns_iterator = User.__table__.columns.__iter__()
for column in columns_iterator:
    print(f"Column: {column.name}, Type: {column.type}, Doc: {column.doc}, Nullable: {column.nullable}, Primary Key: {column.primary_key}, Foreign Key: {column.foreign_keys}")

Column: id, Type: INTEGER, Doc: Unique user identifier, Nullable: False, Primary Key: True, Foreign Key: set()
Column: username, Type: VARCHAR(50), Doc: Username of the user, Nullable: False, Primary Key: False, Foreign Key: set()
Column: email, Type: VARCHAR(100), Doc: User's email address, Nullable: False, Primary Key: False, Foreign Key: set()
Column: created_at, Type: DATETIME, Doc: Datetime when the user was created, Nullable: False, Primary Key: False, Foreign Key: set()
Column: user_role, Type: VARCHAR(5), Doc: Role of the user in the system, Nullable: False, Primary Key: False, Foreign Key: set()


In [52]:
from sqlalchemy.orm import class_mapper
from pprint import pprint
mapper = class_mapper(User)
for col in mapper.columns:
    pprint(col.type.python_type.__name__)


'int'
'str'
'str'
'datetime'
'UserRole'


In [53]:
for relationship in User.__mapper__.relationships:
    print(f"Relationship: {relationship.key}, Mapper: {relationship.mapper}, Direction: {relationship.direction}, Doc: {relationship.doc}, Type: {relationship.mapper.local_table}")

Relationship: posts, Mapper: Mapper[Post(posts)], Direction: RelationshipDirection.ONETOMANY, Doc: List of posts created by the user, Type: posts


In [54]:
for relationship in User.__mapper__.relationships:
    print(dir(relationship.mapper))
    break

['__annotations__', '__class__', '__class_getitem__', '__clause_element__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_acceptable_polymorphic_identities', '_adapt_inherited_property', '_add_with_polymorphic_subclass', '_all_column_expressions', '_all_pk_cols', '_all_tables', '_assert_no_memoizations', '_cache_key_traversal', '_canload', '_check_configure', '_clause_adapter', '_cols_by_table', '_columns_plus_keys', '_columntoproperty', '_compiled_cache', '_compiled_cache_size', '_configure_class_instrumentation', '_configure_failed', '_configure_inheritance', '_configure_pks', '_configure_polymorphic_setter', '_configure_properties', '_co

In [55]:
for relationship in User.__mapper__.relationships:
    print(f"Relationship: {relationship.key}, Related class: {relationship.mapper.class_.__name__}")

Relationship: posts, Related class: Post


In [56]:
mapper = class_mapper(User)

for column in mapper.columns:
    python_type = column.type.python_type if hasattr(column.type, 'python_type') else column.type
    print(f"Column: {column.name}, Python type: {type(python_type)}, Doc: {column.doc}, "
            f"Nullable: {column.nullable}, Primary Key: {column.primary_key}, "
            f"Foreign Key: {[fk.column.table.name for fk in column.foreign_keys]}")

Column: id, Python type: <class 'type'>, Doc: Unique user identifier, Nullable: False, Primary Key: True, Foreign Key: []
Column: username, Python type: <class 'type'>, Doc: Username of the user, Nullable: False, Primary Key: False, Foreign Key: []
Column: email, Python type: <class 'type'>, Doc: User's email address, Nullable: False, Primary Key: False, Foreign Key: []
Column: created_at, Python type: <class 'type'>, Doc: Datetime when the user was created, Nullable: False, Primary Key: False, Foreign Key: []
Column: user_role, Python type: <class 'enum.EnumType'>, Doc: Role of the user in the system, Nullable: False, Primary Key: False, Foreign Key: []


In [57]:
for column in Post.__table__.columns:
    for fk in column.foreign_keys:
        related_table = fk.column.table
        # Find the model class for the related table
        for cls in Base.__subclasses__():
            if hasattr(cls, '__tablename__') and cls.__tablename__ == related_table.name:
                print(f"Foreign key '{fk}' refers to model: {cls.__name__}")

Foreign key 'ForeignKey('users.id')' refers to model: User


In [58]:
from sqlalchemy.orm import RelationshipDirection
type(RelationshipDirection.MANYTOONE)  # This will return the type of the RelationshipDirection enum

<enum 'RelationshipDirection'>

In [59]:
from sqlalchemy.schema import ForeignKey
f = ForeignKey('users.id')
print(f)

ForeignKey('users.id')


In [60]:
repr(Post.__doc__)

'None'

In [61]:
def _get_model_name_by_table_name(self, table_name: str) -> str:
    """
    Returns the model name based on the table name.
    """
    base = next(self.model_class.__bases__)
    for cls_ in base.__subclasses__():
        if hasattr(cls_, "__tablename__") and cls_.__tablename__ == table_name:
            return cls_.__name__
    return "UnknownModel"

In [62]:
base = Post.__bases__[0]
base.__subclasses__()

[__main__.User, __main__.Post]

In [63]:
for cls_ in base.__subclasses__():
    if hasattr(cls_, "__tablename__") and cls_.__tablename__ == "users":
        print(cls_.__name__)

User


In [64]:
mapper = class_mapper(User)

for column in mapper.columns:
    python_type = f"{column.type.python_type.__name__.capitalize() if hasattr(column.type, 'python_type') else column.type}({column.type.length if hasattr(column.type, 'length') and column.type.length else ''})"
    print(f"Column: {column.name}, Python type: {python_type}, Doc: {column.doc}, "
            f"Nullable: {column.nullable}, Primary Key: {column.primary_key}, "
            f"Foreign Key: {[fk.column.table.name for fk in column.foreign_keys]} "
            f"Default: {column.server_default.arg if column.server_default else 'None'} "
            f"Unique: {column.unique}")

Column: id, Python type: Int(), Doc: Unique user identifier, Nullable: False, Primary Key: True, Foreign Key: [] Default: None Unique: None
Column: username, Python type: Str(50), Doc: Username of the user, Nullable: False, Primary Key: False, Foreign Key: [] Default: None Unique: True
Column: email, Python type: Str(100), Doc: User's email address, Nullable: False, Primary Key: False, Foreign Key: [] Default: None Unique: True
Column: created_at, Python type: Datetime(), Doc: Datetime when the user was created, Nullable: False, Primary Key: False, Foreign Key: [] Default: None Unique: None
Column: user_role, Python type: Userrole(5), Doc: Role of the user in the system, Nullable: False, Primary Key: False, Foreign Key: [] Default: user Unique: None


In [65]:
Post.__bases__

(__main__.Base,)

In [67]:
mapper = class_mapper(Post)

for column in mapper.columns:
    print(column.type.python_type)

<class 'int'>
<class 'str'>
<class 'str'>
<class 'int'>
<class 'list'>
